In [22]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder \
  .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
  .getOrCreate()

rdd = spark.sparkContext.parallelize([
('APPN','2021-06-14',114.82,117.65,114.19,119.78,117.65,1117338.0,'2021-06-14'),
('APPL','2021-06-14',111.22,90.0,110.07,114.4,113.0,784841.0,'2021-06-13'),
('APPN','2021-06-11',111.22,113.0,110.07,114.4,113.0,788500.0,'2021-06-12'),
('APPL','2021-06-11',114.82,83.0,114.19,119.78,117.65,1117338.0,'2021-06-14'),    
('APPN','2021-06-10',107.05,111.04,104.61,111.38,111.04,1018900.0,'2021-05-12'),
('APPN','2021-06-09',112.66,107.37,105.4,112.97,107.37,1362000.0,'2021-05-12')
])

df = spark.createDataFrame(rdd, ["Name","Date","Open","Close","Low","High","AdjClose","Volume","record_ingestion_date"])

my_window = Window.partitionBy("Name").orderBy("Date")

df = df.withColumn("prev_close", F.lag(df.Close).over(my_window))
df = df.withColumn("change", F.when(F.isnull(df.Close - df.prev_close), 0)
                              .otherwise(F.round((df.Close - df.prev_close),2)))
df = df.withColumn("change_in_percent",F.round((df.change/df.prev_close)*100,2))

df.show()

+----+----------+------+------+------+------+--------+---------+---------------------+----------+------+-----------------+
|Name|      Date|  Open| Close|   Low|  High|AdjClose|   Volume|record_ingestion_date|prev_close|change|change_in_percent|
+----+----------+------+------+------+------+--------+---------+---------------------+----------+------+-----------------+
|APPL|2021-06-11|114.82|  83.0|114.19|119.78|  117.65|1117338.0|           2021-06-14|      null|   0.0|             null|
|APPL|2021-06-14|111.22|  90.0|110.07| 114.4|   113.0| 784841.0|           2021-06-13|      83.0|   7.0|             8.43|
|APPN|2021-06-09|112.66|107.37| 105.4|112.97|  107.37|1362000.0|           2021-05-12|      null|   0.0|             null|
|APPN|2021-06-10|107.05|111.04|104.61|111.38|  111.04|1018900.0|           2021-05-12|    107.37|  3.67|             3.42|
|APPN|2021-06-11|111.22| 113.0|110.07| 114.4|   113.0| 788500.0|           2021-06-12|    111.04|  1.96|             1.77|
|APPN|2021-06-14